In [40]:
import pandas as pd
from imblearn.over_sampling import SMOTE

In [41]:
df = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\new_data\2014CMDS_Negative_ITEMS.xlsx')
df.head()

,gender,age,nation,edu,Hukou_nature,marital status,MacArthur1,MacArthur2,MacArthur3,MacArthur4,...,SWLS-1,SWLS-2,SWLS-3,SWLS-4,SWLS-5,PSS-4-1,PSS-4-2,PSS-4-3,PSS-4-4,Negative
0,1,-1.22626,1,3,1,0,-1.09450,-0.91834,-0.38444,-1.39294,...,-1.02085,-0.93843,-1.13542,-0.84271,-0.58104,-0.12589,-1.39873,-1.31589,-0.14281,0
1,1,-1.45566,1,3,1,0,-1.70635,-1.54229,-0.98589,-1.99939,...,-1.70821,-1.65829,-1.82634,-1.47552,-1.11822,-0.12589,-1.39873,-1.31589,-0.14281,0
2,2,1.75600,1,2,1,1,-1.70635,-1.54229,-0.98589,-1.99939,...,-1.02085,-0.93843,-0.44450,-0.84271,-0.04385,-0.12589,-1.39873,-1.31589,-0.14281,1
3,1,-1.68507,1,3,1,0,-1.70635,-1.54229,-0.98589,-1.99939,...,-1.02085,-0.93843,-1.13542,-0.84271,-0.58104,-0.12589,-1.39873,-1.31589,-0.14281,1
4,1,-1.57037,1,3,1,0,-2.31820,-2.16623,-1.58734,-2.60584,...,-1.02085,-0.93843,-1.13542,-0.84271,-0.58104,-1.27949,-2.27990,-2.29382,-1.32543,1


In [42]:
X = df.drop('Negative', axis = 1)
X.shape

(15997, 27)

In [43]:
y=df['Negative']
y.shape

(15997,)

In [44]:
pd.Series(y).value_counts()

Negative
0    12990
1     3007
Name: count, dtype: int64

In [45]:
smote = SMOTE()

In [46]:
X.dtypes

gender              int64
age               float64
nation              int64
edu                 int64
Hukou_nature        int64
marital status      int64
MacArthur1        float64
MacArthur2        float64
MacArthur3        float64
MacArthur4        float64
MacArthur5        float64
MacArthur6        float64
SF-6D-1           float64
SF-6D-2           float64
SF-6D-3           float64
SF-6D-4           float64
SF-6D-5           float64
SF-6D-6           float64
SWLS-1            float64
SWLS-2            float64
SWLS-3            float64
SWLS-4            float64
SWLS-5            float64
PSS-4-1           float64
PSS-4-2           float64
PSS-4-3           float64
PSS-4-4           float64
dtype: object

In [47]:
X = X.apply(pd.to_numeric, errors='coerce')

In [48]:
X.dtypes

gender              int64
age               float64
nation              int64
edu                 int64
Hukou_nature        int64
marital status      int64
MacArthur1        float64
MacArthur2        float64
MacArthur3        float64
MacArthur4        float64
MacArthur5        float64
MacArthur6        float64
SF-6D-1           float64
SF-6D-2           float64
SF-6D-3           float64
SF-6D-4           float64
SF-6D-5           float64
SF-6D-6           float64
SWLS-1            float64
SWLS-2            float64
SWLS-3            float64
SWLS-4            float64
SWLS-5            float64
PSS-4-1           float64
PSS-4-2           float64
PSS-4-3           float64
PSS-4-4           float64
dtype: object

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [50]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [51]:
print(y_train.value_counts())  # 原始训练集的类别分布
print(y_train_resampled.value_counts())  # 过采样后的类别分布

Negative
0    10405
1     2392
Name: count, dtype: int64
Negative
0    10405
1    10405
Name: count, dtype: int64


In [52]:
df2=pd.concat([y_train_resampled, X_train_resampled], axis=1)

In [53]:
df2.to_excel(r'C:\Users\Lenovo\Desktop\ML_health\new_data\2014CMDS_Negative_ITEMS_smote.xlsx',index = False)

In [54]:
df3 = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\new_data\2014CMDS_Negative_ITEMS_smote.xlsx')

In [55]:
X = df3.drop('Negative', axis = 1)
X.shape

(20810, 27)

In [56]:
y=df3['Negative']
y.shape

(20810,)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [7]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [8]:
param_grid = {
    'n_estimators': [200],
    'learning_rate': [0.05],
    'num_leaves': [31]
}

# 创建 LightGBM 分类器
gbm = LGBMClassifier()

# 创建网格搜索对象
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 最佳参数和最佳模型
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6678, number of negative: 6636
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5665
[LightGBM] [Info] Number of data points in the train set: 13314, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501577 -> initscore=0.006309
[LightGBM] [Info] Start training from score 0.006309
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6678, number of negative: 6636
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead o

In [9]:
y_pred = best_model.predict(X_val)
# 验证集的真实标签
y_true = y_val

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = best_model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2107
           1       1.00      1.00      1.00      2054

    accuracy                           1.00      4161
   macro avg       1.00      1.00      1.00      4161
weighted avg       1.00      1.00      1.00      4161

[[2107    0]
 [   0 2054]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC: 1.0
Specificity: 1.0


In [11]:
# 获取特征重要性
feature_importances = best_model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

           Feature  Importance
1              age        1153
12         SF-6D-1         521
11      MacArthur6         397
4     Hukou_nature         368
10      MacArthur5         364
7       MacArthur2         304
6       MacArthur1         271
9       MacArthur4         257
8       MacArthur3         243
19          SWLS-2         229
20          SWLS-3         209
31           Tired         200
30             Sad         200
29           Annoy         200
27         Nervous         200
32       Worthless         198
28       Desperate         155
15         SF-6D-4         139
18          SWLS-1         136
22          SWLS-5          71
17         SF-6D-6          43
23         PSS-4-1          43
21          SWLS-4          21
16         SF-6D-5          17
3              edu          15
25         PSS-4-3          13
26         PSS-4-4           8
14         SF-6D-3           7
0           gender           4
24         PSS-4-2           3
5   marital status           2
13      

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [13]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [14]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2107
           1       1.00      1.00      1.00      2054

    accuracy                           1.00      4161
   macro avg       1.00      1.00      1.00      4161
weighted avg       1.00      1.00      1.00      4161

[[2107    0]
 [   0 2054]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC: 1.0
Specificity: 1.0


In [16]:
# 获取特征重要性
feature_importances = model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

           Feature  Importance
29           Annoy    0.287759
27         Nervous    0.260524
31           Tired    0.157598
30             Sad    0.107433
26         PSS-4-4    0.035505
32       Worthless    0.030216
23         PSS-4-1    0.028374
28       Desperate    0.018524
25         PSS-4-3    0.009955
24         PSS-4-2    0.006088
19          SWLS-2    0.005993
20          SWLS-3    0.005599
16         SF-6D-5    0.004993
22          SWLS-5    0.004437
14         SF-6D-3    0.004391
21          SWLS-4    0.003327
1              age    0.003280
8       MacArthur3    0.002736
17         SF-6D-6    0.002718
18          SWLS-1    0.002415
9       MacArthur4    0.002255
7       MacArthur2    0.002165
11      MacArthur6    0.002030
12         SF-6D-1    0.001958
6       MacArthur1    0.001793
10      MacArthur5    0.001784
13         SF-6D-2    0.001492
3              edu    0.001271
15         SF-6D-4    0.001213
4     Hukou_nature    0.000712
0           gender    0.000639
5   mari